## Mathematical Formulation

### 1. Sample Skewness

$$
\text{skew}(x) =
\begin{cases}
0, & n < 3 \text{ or } \sigma = 0, \\[6pt]
\dfrac{\frac{1}{n} \sum_{i=1}^n (x_i - \mu)^3}{\sigma^3}, & \text{otherwise.}
\end{cases}
\tag{1}
$$

$$
\mu = \frac{1}{n} \sum_{i=1}^n x_i, \quad
\sigma = \sqrt{\frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2}.
\tag{2}
$$


### 2. Skew-Aware Score Adjustment

$$
a = -\tanh\!\big(\text{skew}(P) \cdot \lambda_s\big) \cdot \lambda_c
\tag{3}
$$

$$
P' = \mathrm{clip}(P + a, f, c)
\tag{4}
$$

where $ \lambda_s $ is the skew sensitivity, $ \lambda_c $ is the clipping strength, and $(f, c)$ are the lower and upper bounds (typically $0$ and $1$).


### 3. High-Confidence Score–Frequency Blending

$$
M_h = \mathbb{I}(P > p_h) \cdot \mathbb{I}(R > f_h), \quad
M_l = \mathbb{I}(P < 1 - p_h) \cdot \mathbb{I}(R < 1 - f_h)
\tag{5}
$$

$$
A_h = \min(P, R) \cdot M_h, \quad
A_l = \min(1 - P, 1 - R) \cdot M_l
\tag{6}
$$

$$
S_{\text{adj}} = P + \alpha A_h - \beta A_l
\tag{7}
$$

$$
S = (1 - \gamma) P + \gamma S_{\text{adj}}
\tag{8}
$$

$$
S' = \mathrm{clip}(S, 0, 1)
\tag{9}
$$


### 4. Full Pipeline

$$
\text{blend} = \text{combined\_score\_sample\_blend}(P, F_{\text{raw}})
\tag{10}
$$

$$
\text{score\_adj} = \text{combined\_score\_distribution\_aware\_simple}(\text{blend})
\tag{11}
$$


### 5. Variable Definitions

| Symbol | Meaning |
|:--------|:---------|
| $x_i$ | Individual data sample |
| $n$ | Number of samples |
| $\mu$ | Mean of samples |
| $\sigma$ | Standard deviation |
| $P$ | Base model score (probability) |
| $F_{\text{raw}}$ | Raw clone frequency |
| $R$ | Percentile-normalized frequency, $R = F_{\text{percentile}}(F_{\text{raw}})$ |
| $p_h, f_h$ | High-confidence thresholds for $P$ and $R$ |
| $M_h, M_l$ | Masks for high and low confidence regions |
| $A_h, A_l$ | Adjustment terms for high and low confidence |
| $\alpha, \beta, \gamma$ | Blend coefficients controlling balance and smoothing |
| $\lambda_s$ | Skew sensitivity (scaling factor) |
| $\lambda_c$ | Clipping limit for skew adjustment |
| $(f, c)$ | Lower and upper clipping bounds |
| $a$ | Global skew-based adjustment |
| $S, S_{\text{adj}}, S'$ | Intermediate and final adjusted scores |
| $\mathrm{clip}(\cdot)$ | Truncation to a specified range (e.g. $[0, 1]$) |
| $\mathbb{I}(\cdot)$ | Indicator function (1 if condition true, else 0) |


## Intuitive Explanation of the Inverse Logit Mean

The **inverse logit mean** is a way to average probabilities more accurately when they come from models that use a *logit (log-odds)* scale internally.

In simple terms:

1. **Convert all probabilities to the logit scale** —  
   this turns curved probability values (between 0 and 1) into linear "log-odds" values (from −∞ to +∞).

2. **Take the average (mean) of these logit values** —  
   since logits are linear, averaging here makes statistical sense.

3. **Convert the mean logit back to probability space** using the inverse logit (sigmoid) function —  
   this gives a single representative probability on the familiar 0–1 scale.

Mathematically:

$$
\text{logit}(p) = \log\!\left(\frac{p}{1 - p}\right)
\tag{1}
$$

$$
\text{invlogit}(x) = \frac{1}{1 + e^{-x}}
\tag{2}
$$

So the **inverse logit mean** of a set of probabilities $\{p_i\}$ is:

$$
\mathrm{ILM}(p_1, p_2, \ldots, p_n)
= \mathrm{invlogit}\!\left( \frac{1}{n} \sum_{i=1}^n \text{logit}(p_i) \right)
\tag{3}
$$

---

### 🔍 Why do this?

Averaging probabilities directly can be misleading because probabilities are *nonlinear*.  
By first moving to the **logit (log-odds)** space, you average on a scale where changes are symmetric and linear.  
Then, converting back with the inverse logit gives a more accurate "average" probability —  
especially when probabilities are close to 0 or 1.

---

**Example intuition:**

If two events have probabilities 0.1 and 0.9:  
- A direct average gives 0.5.  
- But in logit space:  
  $\text{logit}(0.1) = -2.197$, $\text{logit}(0.9) = +2.197$.  
  Their mean logit is $0$, and $\text{invlogit}(0) = 0.5$.  

That one happens to match — but for less symmetric cases (like 0.2 and 0.6),  
the inverse logit mean will better reflect the model’s underlying uncertainty.
